# 🎯 3D物体分类案例 - ModelNet40数据集

> 本案例将使用Hugging Face上的jxie/modelnet40数据集进行3D物体分类任务演示

**目标**：使用ModelNet40数据集进行3D物体分类，包括数据下载、预处理、可视化展示等完整流程。  
**数据集**：ModelNet40 - 包含40个类别的3D CAD模型  
**应用场景**：3D物体识别、机器人导航、AR/VR应用、工业检测等

---

## ✅ 本教程包括
1. 3D物体分类简介
2. 环境配置与依赖安装
3. ModelNet40数据集下载
4. 数据预处理与格式转换
5. 3D数据可视化展示
6. 基础分类模型搭建（后续扩展）

> 注：本案例专注于数据准备和可视化，为后续的深度学习模型训练做准备。


## 一、3D物体分类简介


### 什么是3D物体分类？

3D物体分类是计算机视觉领域的重要任务，旨在识别和分类三维物体。与传统的2D图像分类不同，3D分类需要考虑：

- **几何特征**：物体的形状、大小、拓扑结构
- **空间关系**：物体各部分之间的空间位置关系  
- **多视角信息**：从不同角度观察物体的特征
- **数据表示**：点云、体素、网格等多种3D数据格式

### ModelNet40数据集特点

- **数据规模**：12,311个3D CAD模型
- **类别数量**：40个日常物体类别
- **数据格式**：.off文件（Object File Format）
- **应用价值**：3D深度学习领域的标准基准数据集

### 常见3D数据表示方法

| 表示方法 | 特点 | 优缺点 |
|---------|------|--------|
| **点云 (Point Cloud)** | 无序的3D点集合 | 简单直观，但需要处理排列不变性 |
| **体素 (Voxel)** | 3D网格中的体素表示 | 规则结构，但内存消耗大 |
| **网格 (Mesh)** | 顶点、边、面的组合 | 高效存储，但拓扑复杂 |
| **多视角 (Multi-view)** | 从多个角度渲染的2D图像 | 可利用2D CNN，但丢失3D信息 |


## 二、环境配置与依赖安装


In [8]:
# 安装必要的依赖包
%pip install datasets
%pip install open3d
%pip install matplotlib
%pip install numpy
%pip install plotly
%pip install trimesh


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [9]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from datasets import load_dataset
import os
import requests
from tqdm import tqdm
import zipfile


## 三、ModelNet40数据集下载


In [10]:
# 首先检查数据集的实际结构
print("=== 检查数据集结构 ===")
print(f"数据集类型: {type(dataset)}")
print(f"可用的分割: {list(dataset.keys())}")

# 检查训练集的结构
train_data = dataset['train']
print(f"\n训练集类型: {type(train_data)}")
print(f"训练集大小: {len(train_data)}")

# 查看第一个样本的结构
sample = train_data[0]
print(f"\n样本类型: {type(sample)}")
print(f"样本键: {list(sample.keys()) if hasattr(sample, 'keys') else 'N/A'}")

# 打印样本的详细信息
print(f"\n样本内容:")
for key, value in sample.items():
    print(f"  {key}: {type(value)} - {value if not hasattr(value, 'shape') else f'shape: {value.shape}'}")


=== 检查数据集结构 ===
数据集类型: <class 'datasets.dataset_dict.DatasetDict'>
可用的分割: ['train', 'test']

训练集类型: <class 'datasets.arrow_dataset.Dataset'>
训练集大小: 9843

样本类型: <class 'dict'>
样本键: ['inputs', 'label']

样本内容:
  inputs: <class 'list'> - [[0.3783285319805145, 0.20441561937332153, 0.09830210357904434], [-0.945515513420105, -0.09821636974811554, 0.08101163804531097], [-0.2055421769618988, -0.21534216403961182, -0.9406888484954834], [-0.19596892595291138, -0.12615464627742767, 0.5955250263214111], [-0.3000047504901886, 0.1432054042816162, -0.05222263187170029], [0.9497930407524109, -0.10984102636575699, 0.08433297276496887], [0.05562199652194977, -0.1800774335861206, -0.40614965558052063], [0.17895087599754333, -0.3010129928588867, 0.24981346726417542], [0.1563853621482849, 0.219556987285614, 0.5341781377792358], [0.2604212462902069, -0.19209285080432892, -0.836652934551239], [-0.6580248475074768, 0.2180916965007782, 0.2522556185722351], [0.7682920098304749, 0.23450297117233276, 0.24512453377

In [11]:
# 根据实际数据结构调整数据预处理
def preprocess_point_cloud(points, num_points=1024):
    """
    预处理点云数据
    Args:
        points: 原始点云数据 (N, 3)
        num_points: 目标点数
    Returns:
        处理后的点云数据
    """
    # 确保points是numpy数组
    if not isinstance(points, np.ndarray):
        points = np.array(points)
    
    # 随机采样到固定点数
    if len(points) > num_points:
        indices = np.random.choice(len(points), num_points, replace=False)
        points = points[indices]
    elif len(points) < num_points:
        # 如果点数不足，随机重复采样
        indices = np.random.choice(len(points), num_points, replace=True)
        points = points[indices]
    
    # 归一化到单位球
    centroid = np.mean(points, axis=0)
    points = points - centroid
    max_dist = np.max(np.linalg.norm(points, axis=1))
    if max_dist > 0:  # 避免除零错误
        points = points / max_dist
    
    return points

# 测试数据预处理 - 根据实际数据结构调整
sample_data = dataset['train'][0]
print("原始数据示例:")
print(f"样本键: {list(sample_data.keys())}")

# 根据实际的数据结构来访问点云数据
# 可能的键名: 'points', 'point_cloud', 'data', 'features' 等
points_key = None
for key in sample_data.keys():
    if 'point' in key.lower() or 'data' in key.lower():
        points_key = key
        break

if points_key:
    print(f"找到点云数据键: {points_key}")
    points = sample_data[points_key]
    print(f"点云形状: {points.shape if hasattr(points, 'shape') else len(points)}")
    print(f"标签: {sample_data.get('label', 'N/A')}")
    
    # 预处理示例数据
    processed_points = preprocess_point_cloud(points)
    print(f"\n预处理后点云形状: {processed_points.shape}")
    print(f"点云范围: [{processed_points.min():.3f}, {processed_points.max():.3f}]")
else:
    print("未找到点云数据，显示所有键值对:")
    for key, value in sample_data.items():
        print(f"  {key}: {type(value)}")


原始数据示例:
样本键: ['inputs', 'label']
未找到点云数据，显示所有键值对:
  inputs: <class 'list'>
  label: <class 'int'>


In [12]:
# 更新可视化函数以处理不同的数据结构
def get_points_from_sample(sample):
    """
    从样本中提取点云数据，适应不同的数据结构
    """
    # 尝试不同的可能键名
    possible_keys = ['points', 'point_cloud', 'data', 'features', 'coordinates']
    
    for key in possible_keys:
        if key in sample:
            points = sample[key]
            if hasattr(points, 'shape') and len(points.shape) >= 2:
                return points
    
    # 如果没有找到，返回第一个看起来像点云的数据
    for key, value in sample.items():
        if hasattr(value, 'shape') and len(value.shape) >= 2 and value.shape[1] >= 3:
            return value
    
    return None

# 测试更新后的函数
sample = dataset['train'][0]
points = get_points_from_sample(sample)

if points is not None:
    print(f"成功提取点云数据，形状: {points.shape}")
    
    # 预处理并可视化
    processed_points = preprocess_point_cloud(points)
    
    # 使用Plotly可视化
    fig = visualize_point_cloud_plotly(
        processed_points, 
        title=f"样本可视化 - 标签: {sample.get('label', 'N/A')}"
    )
    fig.show()
    
else:
    print("无法提取点云数据，显示样本结构:")
    for key, value in sample.items():
        print(f"  {key}: {type(value)} - {getattr(value, 'shape', 'N/A')}")


无法提取点云数据，显示样本结构:
  inputs: <class 'list'> - N/A
  label: <class 'int'> - N/A


In [ ]:
# 更新统计分析函数以处理不同的数据结构
def analyze_dataset_statistics_updated(dataset):
    """
    分析数据集的统计信息 - 更新版本
    """
    train_data = dataset['train']
    test_data = dataset['test']
    
    # 统计每个类别的样本数量
    train_labels = []
    test_labels = []
    train_point_counts = []
    test_point_counts = []
    
    print("正在分析训练集...")
    for item in tqdm(train_data):
        # 获取标签
        label = item.get('label', item.get('class', item.get('category', 0)))
        train_labels.append(label)
        
        # 获取点云数据
        points = get_points_from_sample(item)
        if points is not None:
            train_point_counts.append(len(points))
        else:
            train_point_counts.append(0)
    
    print("正在分析测试集...")
    for item in tqdm(test_data):
        # 获取标签
        label = item.get('label', item.get('class', item.get('category', 0)))
        test_labels.append(label)
        
        # 获取点云数据
        points = get_points_from_sample(item)
        if points is not None:
            test_point_counts.append(len(points))
        else:
            test_point_counts.append(0)
    
    train_counts = Counter(train_labels)
    test_counts = Counter(test_labels)
    
    print("=== 数据集统计信息 ===")
    print(f"训练集样本数: {len(train_data)}")
    print(f"测试集样本数: {len(test_data)}")
    print(f"总样本数: {len(train_data) + len(test_data)}")
    print(f"类别数: {len(set(train_labels))}")
    
    if train_point_counts:
        print(f"\n训练集点云大小统计:")
        print(f"  平均点数: {np.mean(train_point_counts):.1f}")
        print(f"  最小点数: {min(train_point_counts)}")
        print(f"  最大点数: {max(train_point_counts)}")
        print(f"  标准差: {np.std(train_point_counts):.1f}")
    
    if test_point_counts:
        print(f"\n测试集点云大小统计:")
        print(f"  平均点数: {np.mean(test_point_counts):.1f}")
        print(f"  最小点数: {min(test_point_counts)}")
        print(f"  最大点数: {max(test_point_counts)}")
        print(f"  标准差: {np.std(test_point_counts):.1f}")
    
    return train_counts, test_counts

# 执行更新后的统计分析
print("执行数据集统计分析...")
train_counts, test_counts = analyze_dataset_statistics_updated(dataset)


执行数据集统计分析...
正在分析训练集...


100%|████████████████████████████████████████████████████| 9843/9843 [03:39<00:00, 44.82it/s]


正在分析测试集...


  9%|████▊                                                | 223/2468 [00:05<01:12, 30.82it/s]

In [ ]:
# 从Hugging Face下载ModelNet40数据集
print("正在从Hugging Face下载ModelNet40数据集...")

# 使用jxie/modelnet40数据集
dataset = load_dataset("jxie/modelnet40")

print("数据集下载完成！")
print(f"数据集结构: {dataset}")
print(f"训练集大小: {len(dataset['train'])}")
print(f"测试集大小: {len(dataset['test'])}")


In [ ]:
# 查看数据集的基本信息
print("=== 数据集基本信息 ===")
print(f"特征字段: {dataset['train'].features}")
print(f"样本示例: {dataset['train'][0]}")

# 查看类别分布
train_labels = [item['label'] for item in dataset['train']]
test_labels = [item['label'] for item in dataset['test']]

print(f"\n训练集标签范围: {min(train_labels)} - {max(train_labels)}")
print(f"测试集标签范围: {min(test_labels)} - {max(test_labels)}")

# 统计每个类别的样本数量
from collections import Counter
train_label_counts = Counter(train_labels)
test_label_counts = Counter(test_labels)

print(f"\n训练集类别分布（前10个）:")
for label, count in train_label_counts.most_common(10):
    print(f"类别 {label}: {count} 个样本")

print(f"\n测试集类别分布（前10个）:")
for label, count in test_label_counts.most_common(10):
    print(f"类别 {label}: {count} 个样本")


## 四、数据预处理与格式转换


In [ ]:
# 定义ModelNet40的40个类别名称
modelnet40_classes = [
    'airplane', 'bathtub', 'bed', 'bench', 'bookshelf', 'bottle', 'bowl', 'car', 'chair',
    'cone', 'cup', 'curtain', 'desk', 'door', 'dresser', 'flower_pot', 'glass_box',
    'guitar', 'keyboard', 'lamp', 'laptop', 'mantel', 'monitor', 'night_stand',
    'person', 'piano', 'plant', 'radio', 'range_hood', 'sink', 'sofa', 'stairs',
    'stool', 'table', 'tent', 'toilet', 'tv_stand', 'vase', 'wardrobe', 'xbox'
]

print("ModelNet40数据集包含的40个类别:")
for i, class_name in enumerate(modelnet40_classes):
    print(f"{i:2d}. {class_name}")

# 创建标签到类别名称的映射
label_to_class = {i: class_name for i, class_name in enumerate(modelnet40_classes)}
class_to_label = {class_name: i for i, class_name in enumerate(modelnet40_classes)}

print(f"\n标签映射示例:")
print(f"标签 0 -> {label_to_class[0]}")
print(f"标签 5 -> {label_to_class[5]}")
print(f"标签 39 -> {label_to_class[39]}")


In [ ]:
# 数据预处理函数
def preprocess_point_cloud(points, num_points=1024):
    """
    预处理点云数据
    Args:
        points: 原始点云数据 (N, 3)
        num_points: 目标点数
    Returns:
        处理后的点云数据
    """
    # 随机采样到固定点数
    if len(points) > num_points:
        indices = np.random.choice(len(points), num_points, replace=False)
        points = points[indices]
    elif len(points) < num_points:
        # 如果点数不足，随机重复采样
        indices = np.random.choice(len(points), num_points, replace=True)
        points = points[indices]
    
    # 归一化到单位球
    centroid = np.mean(points, axis=0)
    points = points - centroid
    max_dist = np.max(np.linalg.norm(points, axis=1))
    points = points / max_dist
    
    return points

# 测试数据预处理
sample_data = dataset['train'][0]
print("原始数据示例:")
print(f"点云形状: {sample_data['points'].shape}")
print(f"标签: {sample_data['label']} -> {label_to_class[sample_data['label']]}")

# 预处理示例数据
processed_points = preprocess_point_cloud(sample_data['points'])
print(f"\n预处理后点云形状: {processed_points.shape}")
print(f"点云范围: [{processed_points.min():.3f}, {processed_points.max():.3f}]")


## 五、3D数据可视化展示


In [ ]:
# 使用Plotly进行3D点云可视化
def visualize_point_cloud_plotly(points, title="3D Point Cloud", colors=None):
    """
    使用Plotly可视化3D点云
    Args:
        points: 点云数据 (N, 3)
        title: 图表标题
        colors: 颜色数组，可选
    """
    if colors is None:
        colors = points[:, 2]  # 使用Z坐标作为颜色
    
    fig = go.Figure(data=[go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1], 
        z=points[:, 2],
        mode='markers',
        marker=dict(
            size=2,
            color=colors,
            colorscale='Viridis',
            opacity=0.8
        ),
        text=[f'Point {i}' for i in range(len(points))],
        hovertemplate='X: %{x}<br>Y: %{y}<br>Z: %{z}<extra></extra>'
    )])
    
    fig.update_layout(
        title=title,
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='cube'
        ),
        width=800,
        height=600
    )
    
    return fig

# 可视化几个不同类别的样本
print("正在生成3D可视化...")

# 选择几个不同类别的样本进行展示
sample_indices = [0, 100, 200, 300, 400]  # 选择5个样本
figs = []

for i, idx in enumerate(sample_indices):
    sample = dataset['train'][idx]
    points = preprocess_point_cloud(sample['points'])
    class_name = label_to_class[sample['label']]
    
    fig = visualize_point_cloud_plotly(
        points, 
        title=f"样本 {i+1}: {class_name} (标签: {sample['label']})"
    )
    figs.append(fig)

# 显示第一个样本
print("显示第一个3D样本:")
figs[0].show()


In [ ]:
# 使用Matplotlib进行2D投影可视化
def visualize_point_cloud_2d(points, title="2D Projection", ax=None):
    """
    使用Matplotlib可视化3D点云的2D投影
    Args:
        points: 点云数据 (N, 3)
        title: 图表标题
        ax: matplotlib轴对象
    """
    if ax is None:
        fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    
    # XY平面投影
    ax.scatter(points[:, 0], points[:, 1], c=points[:, 2], 
               cmap='viridis', s=1, alpha=0.6)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title(title)
    ax.grid(True, alpha=0.3)
    ax.set_aspect('equal')
    
    return ax

# 创建多个子图展示不同类别的样本
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

print("生成2D投影可视化...")

for i, idx in enumerate(sample_indices[:6]):  # 显示前6个样本
    sample = dataset['train'][idx]
    points = preprocess_point_cloud(sample['points'])
    class_name = label_to_class[sample['label']]
    
    visualize_point_cloud_2d(
        points, 
        title=f"{class_name} (标签: {sample['label']})",
        ax=axes[i]
    )

# 隐藏多余的子图
for i in range(len(sample_indices), len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()


In [ ]:
# 数据集统计分析
def analyze_dataset_statistics(dataset):
    """
    分析数据集的统计信息
    """
    train_data = dataset['train']
    test_data = dataset['test']
    
    # 统计每个类别的样本数量
    train_labels = [item['label'] for item in train_data]
    test_labels = [item['label'] for item in test_data]
    
    train_counts = Counter(train_labels)
    test_counts = Counter(test_labels)
    
    # 计算点云大小统计
    train_point_counts = [len(item['points']) for item in train_data]
    test_point_counts = [len(item['points']) for item in test_data]
    
    print("=== 数据集统计信息 ===")
    print(f"训练集样本数: {len(train_data)}")
    print(f"测试集样本数: {len(test_data)}")
    print(f"总样本数: {len(train_data) + len(test_data)}")
    print(f"类别数: {len(set(train_labels))}")
    
    print(f"\n训练集点云大小统计:")
    print(f"  平均点数: {np.mean(train_point_counts):.1f}")
    print(f"  最小点数: {min(train_point_counts)}")
    print(f"  最大点数: {max(train_point_counts)}")
    print(f"  标准差: {np.std(train_point_counts):.1f}")
    
    print(f"\n测试集点云大小统计:")
    print(f"  平均点数: {np.mean(test_point_counts):.1f}")
    print(f"  最小点数: {min(test_point_counts)}")
    print(f"  最大点数: {max(test_point_counts)}")
    print(f"  标准差: {np.std(test_point_counts):.1f}")
    
    return train_counts, test_counts

# 执行统计分析
train_counts, test_counts = analyze_dataset_statistics(dataset)


In [ ]:
# 可视化类别分布
def plot_class_distribution(train_counts, test_counts, top_n=20):
    """
    绘制类别分布图
    """
    # 获取前N个最常见的类别
    common_classes = set(train_counts.keys()) & set(test_counts.keys())
    common_classes = sorted(common_classes)[:top_n]
    
    train_values = [train_counts[cls] for cls in common_classes]
    test_values = [test_counts[cls] for cls in common_classes]
    class_names = [label_to_class[cls] for cls in common_classes]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # 训练集分布
    ax1.bar(range(len(common_classes)), train_values, color='skyblue', alpha=0.7)
    ax1.set_xlabel('类别')
    ax1.set_ylabel('样本数量')
    ax1.set_title('训练集类别分布（前20个）')
    ax1.set_xticks(range(len(common_classes)))
    ax1.set_xticklabels(class_names, rotation=45, ha='right')
    ax1.grid(True, alpha=0.3)
    
    # 测试集分布
    ax2.bar(range(len(common_classes)), test_values, color='lightcoral', alpha=0.7)
    ax2.set_xlabel('类别')
    ax2.set_ylabel('样本数量')
    ax2.set_title('测试集类别分布（前20个）')
    ax2.set_xticks(range(len(common_classes)))
    ax2.set_xticklabels(class_names, rotation=45, ha='right')
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# 绘制类别分布图
print("生成类别分布可视化...")
plot_class_distribution(train_counts, test_counts)


## 六、总结与后续扩展

### 本案例完成的内容

✅ **数据集下载**：成功从Hugging Face下载ModelNet40数据集  
✅ **数据探索**：分析了数据集的基本信息和统计特征  
✅ **数据预处理**：实现了点云数据的标准化和采样  
✅ **3D可视化**：使用Plotly和Matplotlib展示3D点云数据  
✅ **统计分析**：生成了类别分布和数据集统计图表  

### 后续可扩展的方向

1. **深度学习模型**：实现PointNet、PointNet++等3D分类模型
2. **数据增强**：添加旋转、缩放、噪声等数据增强技术
3. **模型训练**：使用PyTorch训练3D物体分类模型
4. **模型评估**：计算准确率、混淆矩阵等评估指标
5. **模型优化**：尝试不同的网络架构和超参数

### 技术要点总结

- **数据格式**：ModelNet40使用点云格式，每个样本包含N×3的坐标矩阵
- **预处理**：归一化和固定点数采样是3D数据预处理的关键步骤
- **可视化**：3D点云可视化有助于理解数据特征和模型行为
- **类别平衡**：不同类别的样本数量可能存在不平衡，需要关注

> 这个案例为3D物体分类任务提供了完整的数据准备和可视化基础，为后续的模型开发奠定了良好的基础。
